In [163]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    Article
)
from paperview.retrieval import pdf_extraction
import tempfile
from io import StringIO
from PIL import Image
import seaborn as sns
import pandas as pd
from io import StringIO

import pdfplumber

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
# url = 'https://www.biorxiv.org/content/10.1101/456574v1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.21.521407v1?rss=1'
url = 'https://www.biorxiv.org/content/10.1101/2022.09.12.507457v1.full'
url = 'https://www.biorxiv.org/content/10.1101/386375v1.full'
# url = 'https://www.biorxiv.org/content/10.1101/2022.07.20.500837v1?rss=1'
url = 'https://www.biorxiv.org/content/10.1101/180125v2.full'
article = Article.from_content_page_url(url, prog_bar=True)

100%|██████████| 42/42 [00:10<00:00,  4.17it/s]


In [ ]:
ii = 3
print(article.data['images'][ii]['image_number'])
print(article.data['images'][ii]['candidate_labels'])
article.data['images'][ii]['image']